In [1]:
from __future__ import annotations as _annotations

from load_models import MODEL
from typing_extensions import TypedDict

import asyncio
from dataclasses import dataclass
from typing import Any, List

import logfire
from httpx import AsyncClient
from pydantic import BaseModel

from pydantic_ai import Agent, RunContext
import nest_asyncio
nest_asyncio.apply()

from pydantic_ai.models.google import GoogleModelSettings



# 'if-token-present' means nothing will be sent (and the example will work) if you don't have logfire configured
logfire.configure(send_to_logfire='if-token-present')
logfire.instrument_pydantic_ai()

#
settings = GoogleModelSettings(google_thinking_config={'include_thoughts': True})


In [2]:
@dataclass
class Deps:
    client: AsyncClient


In [3]:
class WeatherTemp(BaseModel):
    city: str
    weather: str
    temperature: str
    sentiment: str

In [4]:
class LatLng(BaseModel):
    lat: float
    lng: float

In [5]:
weather_agent = Agent(
    name='Weather Agent',
    model=MODEL,
    system_prompt=(
        'Use the `get_lat_lng` tool to get the latitude and longitude of the locations,'
        'then use the `get_weather` tool to get the weather.'
    ),
    deps_type=Deps,
    retries=2,
    output_type= WeatherTemp,
    model_settings=settings
)

In [6]:
@weather_agent.tool
async def get_lat_lng(ctx: RunContext[Deps], location_description: str) -> LatLng:
    """Get the latitude and longitude of a location.

    Args:
        ctx: The context.
        location_description: A description of a location.
    """

    # NOTE: the response here will be random, and is not related to the location description.
    r = await ctx.deps.client.get(
        'https://demo-endpoints.pydantic.workers.dev/latlng',
        params={'location': location_description},
    )

    r.raise_for_status()

    return LatLng.model_validate_json(r.content)

In [7]:
@weather_agent.tool
async def get_weather(ctx: RunContext[Deps], lat: float, lng: float) -> dict[str, Any]:
    """Get the weather at a location.
        Give a descriptive sentiment.

    Args:
        ctx: The context.
        lat: Latitude of the location.
        lng: Longitude of the location.
    """

    # NOTE: the responses here will be random, and are not related to the lat and lng.
    temp_response, descr_response = await asyncio.gather(
        ctx.deps.client.get(
            'https://demo-endpoints.pydantic.workers.dev/number',
            params={'min': 10, 'max': 30},
        ),
        ctx.deps.client.get(
            'https://demo-endpoints.pydantic.workers.dev/weather',
            params={'lat': lat, 'lng': lng},
        ),
    )

    # raise error if not found
    temp_response.raise_for_status()
    descr_response.raise_for_status()

    output= {
        'temperature': f'{temp_response.text} °C',
        'description': descr_response.text,
    }
    return output

In [8]:
async def main(question):
    async with AsyncClient() as client:
        # logfire.instrument_httpx(client, capture_all=True)
        deps = Deps(client=client)
        result = await weather_agent.run(
            question, 
            deps=deps,
            output_type=List[WeatherTemp]  
        )

        print('\nResponse:\n')
        # The output is now a list, so we should loop through it
        for report in result.output:
            print(report)
            print(report.model_dump())
            print('---')


if __name__ == '__main__':
    await main(question = 'What is the weather like in London and Bangalore?')

22:24:35.475 Weather Agent run
22:24:35.484   chat gemini-1.5-flash
22:24:36.482   running 2 tools
22:24:36.484     running tool: get_lat_lng
22:24:36.485     running tool: get_lat_lng
22:24:36.638   chat gemini-1.5-flash
22:24:37.183   running 1 tool
22:24:37.184     running tool: get_weather
22:24:37.234   chat gemini-1.5-flash
22:24:37.747   running 1 tool
22:24:37.748     running tool: get_weather
22:24:37.795   chat gemini-1.5-flash

Response:

city='London' weather='partly cloudy' temperature='11 °C' sentiment='pleasant'
{'city': 'London', 'weather': 'partly cloudy', 'temperature': '11 °C', 'sentiment': 'pleasant'}
---
city='Bangalore' weather='drizzling' temperature='11 °C' sentiment='pleasant'
{'city': 'Bangalore', 'weather': 'drizzling', 'temperature': '11 °C', 'sentiment': 'pleasant'}
---
